In [82]:
import os
from pydantic import Extra
import requests
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv

from langchain.prompts import PromptTemplate
# Run chain
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS # type: ignore
from langchain_ollama import OllamaEmbeddings # type: ignore
import warnings
warnings.filterwarnings('ignore')

In [116]:

load_dotenv()
#os.environ["token"] = "Bearer token|123675a6-95f6-4fb7-bc95-30095472ae3a|02de311fd83421a7fd637bf34dc8f959caa29f39888d2919e4b9640a2220224b"
token = os.getenv("TOKEN2")

In [149]:
#variables and endpoints
client_id = os.getenv("CLIENT_ID")
DatasetId = os.getenv("DATASETID")
dtname = os.getenv("DTNAME")
DatasetID2 = os.getenv("DATASETID2")
dtname2 = os.getenv("DTNAME2")

# Define the API endpoint for interacting with the agents API
agents_endpoint = f"https://api.lab45.ai/v1.1/agents"  # The endpoint where agent-related operations are available

# Define the API endpoint for interacting with the agents chatAPI
agents_endpoint2 = f"https://api.lab45.ai/v1.1/agent_chat_session/query"  # The endpoint where agent-related operations are available

# Define the API endpoint for preparing the skill, specifically for document completion
prepare_endpoint = f"https://api.lab45.ai/v1.1/skills/doc_completion/prepare"

# Define the API endpoint for querying the document completion skill
query_endpoint = f"https://api.lab45.ai/v1.1/skills/doc_completion/query"

# Set the headers for the request, including content type, accepted response format, and authorization token
headers = {
    'Content-Type': "application/json",  # The content type is set to JSON, meaning the body will be JSON-encoded
    'Accept': "text/event-stream, application/json",  # The client expects either event-stream or JSON as the response format
    'Authorization': token  # Replace <api_key> with your actual API key for authentication
}


In [150]:
DatasetID2

'617b94b4-65b9-4b93-b465-2a3c8b97bf66'

In [84]:
from typing import ClassVar

parser = StrOutputParser()

class LlamaLLM(LLM):
    llm_url: ClassVar[str] = 'https://api.lab45.ai/v1.1/skills/completion/query'
    
    backend:        Optional[str]   = 'gpt-35-turbo-16k'
    temp:           Optional[float] = 0.7
    top_p:          Optional[float] = 0.1
    top_k:          Optional[int]   = 40
    n_batch:        Optional[int]   = 8
    n_threads:      Optional[int]   = 4
    n_predict:      Optional[int]   = 256
    max_tokens:     Optional[int]   = 256
    repeat_last_n:  Optional[int]   = 64
    repeat_penalty: Optional[float] = 1.18

    class Config:
        extra = Extra.forbid

    @property
    def _llm_type(self) -> str:
        return "gpt-35-turbo-16k"
    
    @property
    def _get_model_default_parameters(self):
        return {
            "max_tokens": self.max_tokens,
            #"n_predict": self.n_predict,
            "top_k": self.top_k,
            "top_p": self.top_p,
            "temperature": self.temp,
            #"n_batch": self.n_batch,
            #"repeat_penalty": self.repeat_penalty,
            #"repeat_last_n": self.repeat_last_n,
        }

    def _call(
        self,
        prompt: str,
        user: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        payload = {
        "messages": [
            {
            "content": prompt,
            "role": user
            }
        ],
        "skill_parameters": {
            "model_name": "gpt-35-turbo-16k",
            "max_output_tokens": 4096,
            "temperature": 0,
            "top_k": 5
        },
        "stream_response": False
        }

        headers = {"Content-Type": "application/json","Authorization": token}

        response = requests.post(self.llm_url, json=payload, headers=headers, verify=False)

       # print("API Response:", response.json())
        response.raise_for_status()

        return response.json()  # get the response from the API

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            "llmUrl": self.llm_url,
            'model_parameters': self._get_model_default_parameters
            }

In [85]:
llm = LlamaLLM()

In [72]:
skill1="Core Java L1"
skill2="Spring Boot L2"
skill3="Hibernate L3"
skill4="Selenium L4"
title="Senior Java Full Stack Developer"
user = "system"
prompt = """As a skilled Technical Interview Panel with advanced knowledge in technology and data science, your role is to meticulously evaluate a employer job description and provide detailed job description having below points outlined.
            Use the mentioned skills & title to generate the job description, where L1 means Beginner, L2 means Intermediate, L3 means Advanced, and L4 means Expert,
            1. **Job Title and Summary**:
            - "What are the key responsibilities?"
            
            2. **Key Responsibilities**:
            - "List the main roles and responsibilities as per title and combination of skill along with proficiency"

            3. **Qualifications and Skills**:
            - "What qualifications are required?"
            - "List the essential skills needed and mention the level of expertise required."

            4. **Experience Requirements**:
            - "How many years of experience are needed?"
            - "What type of previous experience is beneficial?"

            """ + skill1 + skill2 + skill3 + skill4 + title
prompt2 =   """
            Use the mentioned skills & title to generate the job description, where L1 means Beginner, L2 means Intermediate, L3 means Advanced, and L4 means Expert,
            Job Title and Summary:
            "What are the key responsibilities?"
            Key Responsibilities:
            "List the main roles and responsibilities as per title and combination of skill along with proficiency"
            Qualifications and Skills:
            "What qualifications are required?"
            "List the essential skills needed and mention the level of expertise required."
            Experience Requirements:
            "How many years of experience are needed?"
            "What type of previous experience is beneficial?"
            """ + skill1 + skill2 + skill3 + skill4 + title

In [73]:
#Testing
#prompt = "What is the current date and time?"
result = llm._call(prompt,user)
parsed_result = result['data']['content'] # type: ignore
parser.invoke(parsed_result)


"**Job Title and Summary**:\n- Job Title: Senior Java Full Stack Developer\n- Summary: We are looking for a Senior Java Full Stack Developer to join our team and be responsible for the development and maintenance of our web applications. The ideal candidate should have a strong background in Core Java, Spring Boot, Hibernate, and Selenium.\n\n**Key Responsibilities**:\n- Designing, developing, and maintaining high-quality web applications using Core Java, Spring Boot, Hibernate, and Selenium.\n- Collaborating with cross-functional teams to define, design, and ship new features.\n- Writing clean, efficient, and maintainable code.\n- Troubleshooting and debugging applications to optimize performance.\n- Implementing security and data protection measures.\n- Integrating front-end elements built by front-end developers with server-side logic.\n- Developing and maintaining APIs.\n- Conducting software analysis, testing, and debugging.\n- Keeping up-to-date with the latest industry trends an

In [8]:
prompt2 = "Summarize it into 8 to 10 sentences keeping all the key resposibilities and skills profiencies mentioned in the job description."
result2 = llm._call(prompt,user)
parsed_result2 = result2['data']['content'] # type: ignore
parser.invoke(parsed_result2)

"**Job Title and Summary**:\n- Job Title: Senior Java Full Stack Developer\n- Summary: We are seeking a highly skilled Senior Java Full Stack Developer to join our team. The ideal candidate will have expertise in Core Java, Spring Boot, Hibernate, and Selenium to develop and maintain high-quality web applications.\n\n**Key Responsibilities**:\n- Designing, developing, and maintaining high-quality web applications using Core Java, Spring Boot, Hibernate, and Selenium.\n- Collaborating with cross-functional teams to define, design, and ship new features.\n- Writing well-designed, efficient, and testable code.\n- Troubleshooting and debugging applications to optimize performance.\n- Implementing security and data protection measures.\n- Integrating data storage solutions.\n- Staying updated with emerging technologies and industry trends.\n\n**Qualifications and Skills**:\n- Qualifications: Bachelor's degree in Computer Science, Engineering, or a related field.\n- Essential Skills:\n  - Co

In [ ]:
# Define the payload (request body) for the API call to create an agent, which includes agent details and configuration
payload = {
    "allow_all_access": False,
    "dataset_id": DatasetId, 
    "name": "JD_Agentv2", 
    "description": "JD to Role Cluster Identifier", 
    "instructions": "As a skilled Technical Interview Panel with expertise in technology and content writing , your role is to to meticulously \n evaluate skills and title provided and give detailed job description having below points outlined.\n Use the mentioned skills along with proficiency  & title to generate the job description, where L1 means Beginner, L2 means Intermediate, L3 means Advanced and L4 means Expert,\n If Title and skill list is not given the reply the user with this message  I am designed to generate JD based on Title and Skill list given , can you please provide required information\n 1. **Job Title and Summary**:\n- What are the key responsibilities?\n 2. **Key Responsibilities**:\n - Use title and skill proficiency to explain role and resposibilities , dont mention level like (L1), (L2) instead refer if as beginner, intermidiate etc\n 3. **Qualifications and Skills**:\n - Skill with highest proficiency should be given more relavance and rest skill as per proficency can be reference accordingly\n 4. **Experience Requirements**:\n - How many years of experience are needed?\n - What type of previous experience is beneficial?\n - Use proficincy mentioned in skill to explain experience\n Once JD is created ,update the JD to latest trends in the market\nand later use web tool provided to fetch latest information/updates on this JD and update accordingly\n Finally if user ask to summarise the JD the summarise it in 8 to 10 sentinces keeing title , skills with proficiency information intact", 
    "max_output_tokens": 256,
    "model_name": "gpt-4", 
    "temperature": 0.3,
    #"tools": ["BingSearchTool"],
    "top_k": 5,
    #"type": "Toolset"
    "type" : "Dataset",
    
}

response = requests.post(agents_endpoint, headers=headers, json=payload)

# Print the response from the API call to inspect the result (status code, content, etc.)
print(response.json())

#'9841fb1c-81f0-4481-aeb6-b77bc7f7ea49' JD Agent2
#'8dea253e-4cae-467b-88a5-88df4477c79f'
#'owners': ['bf955f58-9ed4-4d5a-9b49-971fc185e5a4']
#'tenant_id': 'a919164d-8b7c-43fb-8119-f1997d45ca4f'

{'_id': '9841fb1c-81f0-4481-aeb6-b77bc7f7ea49', 'allow_all_access': False, 'dataset_id': '750d37e1-f77f-4e7a-841a-af8e0f5e1375', 'description': 'JD to Role Cluster Identifier', 'instructions': 'As a skilled Technical Interview Panel with expertise in technology and content writing , your role is to to meticulously \n evaluate skills and title provided and give detailed job description having below points outlined.\n Use the mentioned skills along with proficiency  & title to generate the job description, where L1 means Beginner, L2 means Intermediate, L3 means Advanced and L4 means Expert,\n If Title and skill list is not given the reply the user with this message  I am designed to generate JD based on Title and Skill list given , can you please provide required information\n 1. **Job Title and Summary**:\n- What are the key responsibilities?\n 2. **Key Responsibilities**:\n - Use title and skill proficiency to explain role and resposibilities , dont mention level like (L1), (L2) inste

In [19]:
# Define the payload (request body) for the API call, which contains the conversation details and instructions for the agent
payload = {
    "conversation_id": "",  
    "messages": [  
        {
            "content": "I have a job description and I want to enhance it with the latest updates. Can you help me looking in below content and provide me the latest updates?" + prompt2,  
            "name": "Suman",
            "role": "user"
        }
    ],
    "party_id": client_id,  # The unique ID of the party (here it is the agent_id which is generated in agents api)
    "party_type": "Agent",
    "save_conversation": False,  
    "stream_response": False  
}


response = requests.post(agents_endpoint2, headers=headers, json=payload)

# Print the response from the API call to inspect the result (status code, content, etc.)
print(response.json())

{'data': {'name': 'JD_Agent', 'role': 'function', 'content': 'I am designed to generate Job Description based on Title and Skill list given, can you please provide required information?', 'conversation_id': '67ab362910fa7e2d69e30b5d', 'response_status': 'Completed'}}


In [164]:
instruction = """As an AI assistant analyzing role clusters, examine the provided job description or skills and:

1. **Cluster Matching**:
   - Match the input against available role clusters in the dataset
   - Rate the match relevancy as a percentage (0-100%)

2. **Required Output Format**:
   - Cluster Name: [Exact cluster name from dataset]
   - Job Grade: [Grade level if available]
   - Skills Required: [List of skills from cluster along with Proficiencies]
   - Match Percentage: [Calculated relevancy %]
   - Practice Type: [e.g., WIPRO PRACTICE]
   - Classification: [e.g., PREMIUM, ULTRA PREMIUM]

3. **Matching Rules**:
   - Consider skill proficiency levels (L1, L2, L3, L4)
   - Match role titles and their variations
   - Consider skill combinations and their relevance
   - Factor in experience level requirements

4. **Additional Information**:
   - Include any specialized certifications required
   - Note any premium/ultra-premium classification
   - List related clusters if exact match < 80%

Please provide comprehensive cluster information based on the input job description or skills."""

In [77]:
instruction = """As a skilled Technical Interview Panel with expertise in technology and content writing , your role is to to meticulously evaluate skills and title provided and give detailed Job Description having below points outlined.
Use the mentioned skills along with proficiency & title to generate the Job Description, where L1 means Beginner, L2 means Intermediate, L3 means Advanced and L4 means Expert
If Title and skill list is not given the reply the user with this message  I am designed to generate Job Description based on Title and Skill list given , can you please provide required information?
1. **Job Title and Summary**:
- What are the key responsibilities?
2. **Key Responsibilities**:
- Use title and skill proficiency to explain role and resposibilities , dont mention level like (L1), (L2) instead refer if as beginner, intermidiate etc
3. **Qualifications and Skills**:
- Skill with highest proficiency should be given more relavance and rest skill as per proficency can be reference accordingly
4. **Experience Requirements**:
- How many years of experience are needed?
- What type of previous experience is beneficial?
- Use proficincy mentioned in skill to explain experience
Once Job Description is created, update the Job Description with latest trends in the market based on the Web tool to fetch latest information/updates on this Job Description and update highlighting the latest trends
Finally if user ask to summarise the Job Description the summarise it in 8 to 10 sentinces keeing title , skills with proficiency information intact
"""

In [165]:
#Update Instruction to Agent

url = "https://api.lab45.ai/v1.1/agents/" + client_id # type: ignore

payload = {
    "instructions": instruction,
    "dataset_id" : DatasetID2,
    "max_output_tokens": 4000,
    "temperature": 0.2,
    "type": "Dataset",
    "tools": None
    #"tools": ["BingSearchTool"]
}

response = requests.post(url, json=payload, headers=headers)

print(response.json())

{'_id': '8dea253e-4cae-467b-88a5-88df4477c79f', 'allow_all_access': False, 'dataset_id': '617b94b4-65b9-4b93-b465-2a3c8b97bf66', 'instructions': 'As an AI assistant analyzing role clusters, examine the provided job description or skills and:\n\n1. **Cluster Matching**:\n   - Match the input against available role clusters in the dataset\n   - Rate the match relevancy as a percentage (0-100%)\n\n2. **Required Output Format**:\n   - Cluster Name: [Exact cluster name from dataset]\n   - Job Grade: [Grade level if available]\n   - Skills Required: [List of skills from cluster along with Proficiencies]\n   - Match Percentage: [Calculated relevancy %]\n   - Practice Type: [e.g., WIPRO PRACTICE]\n   - Classification: [e.g., PREMIUM, ULTRA PREMIUM]\n\n3. **Matching Rules**:\n   - Consider skill proficiency levels (L1, L2, L3, L4)\n   - Match role titles and their variations\n   - Consider skill combinations and their relevance\n   - Factor in experience level requirements\n\n4. **Additional In

In [166]:
# Define the payload (request body) for the API call, which includes a dataset ID
payload = {
        "dataset_id": DatasetID2  # The ID of the dataset to be used for the document completion task
    }

# Make the POST request to the "prepare" API endpoint with the provided headers and payload
response = requests.post(prepare_endpoint, headers=headers, json=payload)

# Print the response from the API call to see the status or data returned
print(response.json())

{'status': 'Dataset Already in progress , please try after sometime'}


In [80]:
qprompt = "List down premium Clusters..."

In [163]:
# Define the payload (request body) for the API call, which contains the dataset and skill parameters
payload = {
        "dataset_id" : DatasetId,  
        "skill_parameters": {  
            "model_name": "gpt-4", 
            "retrieval_chain": "custom",  
            "emb_type": "openai",  
            "temperature": 0,  
            "max_output_tokens": 1000,  
            "return_sources": False  
        },
        "stream_response": False, 
        "messages": [  
            {"content": "Hi", "role": "user"},  
            {"content": qprompt, "role": "user"} 
        ]
    }


# Make the POST request to the query API endpoint with the provided headers and payload
response = requests.post(query_endpoint, headers=headers, json=payload)

# Print the response from the API call to inspect the status or data returned
print(response.json())

{'data': {'content': 'The premium clusters listed in the document are:\n\n1. **Techno Functional Consultant Network SDN - L1** - Group B3, NETWORK SDN, WIPRO PRACTICE, ULTRA PREMIUM\n2. **Solution Architect Data Visualization - L1** - Group C1, DATA VISUALIZATION, WIPRO PRACTICE, PREMIUM\n3. **Solution Architect .NET Framework - L2** - Group C2, .NET FRAMEWORK, WIPRO PRACTICE, PREMIUM\n4. **Cyber Security Analyst Security Information Event Management - Splunk - L1** - Group A2, SECURITY INFORMATION EVENT MANAGEMENT - SPLUNK, WIPRO PRACTICE, PREMIUM\n5. **Techno Functional Consultant Vulnerability Assessment and Penetration Testing - L2** - Group C1, VULNERABILITY ASSESSMENT AND PENETRATION TESTING, WIPRO PRACTICE, PREMIUM\n6. **Infrastructure Architect Azure-DevOps-Containers - L2** - Group C2, Azure-DevOps-Containers, WIPRO PRACTICE, PREMIUM\n7. **Solution Architect AIOps - Moogsoft - L2** - Group C2, AIOps - Moogsoft, WIPRO PRACTICE, ULTRA PREMIUM\n8. **Technical Lead Automotive Clou

In [162]:
#Uploading the Output file for RAG
url = f"https://api.lab45.ai/v1.1/datasets/{DatasetID2}/ingest"
files = { "files": open('ClusterJD.docx', 'rb') }
response = requests.post(url, files=files, headers=headers)
if response.status_code == 204:
	print("File not uploaded successfully")
else:
	print(response)

File not uploaded successfully
